## 1. Import Packages

In [2]:
import os
import re
import glob
import pandas as pd
import datetime
import geopandas as gpd
from shapely import wkt
from tqdm import tqdm
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

os.chdir(r'C:\Users\senol_c1\Projects\AdProj')

## 2. Read and clean store data, GPS signals data and affinity data

### 2.1. Store Data

In [4]:
#read store data
df_stores = pd.read_csv('stores.csv')

# convert WKT to geometry
df_stores['geometry'] = df_stores['wkt'].apply(wkt.loads)
df_stores_geo = gpd.GeoDataFrame(df_stores, geometry='geometry')
df_stores_geo.drop('wkt', axis=1, inplace=True)

df_stores_geo.head()

,store_id,store_name,geometry
0,place_1,McDonald's,"POLYGON ((13.46119 52.47099, 13.46103 52.47066..."
1,place_2,McDonald's,"POLYGON ((13.46835 52.54716, 13.46841 52.54716..."
2,place_3,McDonald's,"POLYGON ((13.31288 52.41979, 13.31303 52.41966..."
3,place_4,McDonald's,"POLYGON ((13.36358 52.56060, 13.36358 52.56061..."
4,place_5,McDonald's,"POLYGON ((13.32309 52.56135, 13.32307 52.56128..."


### 2.2. GPS Signals Data (Sample Data)

In [5]:
#save data path to a list
path = r'sample_data'              
all_files = glob.glob(path + '\*.csv')

#initialize a dataframe
df_gps_signals_w_stores = pd.DataFrame()

#read files in sample data 
for filename in tqdm(all_files):
    df = pd.read_csv(filename)
    
    # convert utc_timestamps to datetime and get the date only
    df['date'] = pd.to_datetime(df['utc_timestamp'], unit="ms", origin='unix').dt.date
    
    #drop utc_timestamp column
    df.drop(columns = ['utc_timestamp'], axis=1, inplace = True)
    
    #create GeoDataFrame from df Dataframe with geometry from lon and lat columns
    df_gps = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.lon, df.lat))
    
    #drop lon and lan column
    df_gps.drop(columns = ['lon', 'lat'], axis=1, inplace = True)

    #filter gps signals by stores with geopandas -> locate gps signals in store polygons 
    gps_signals_w_stores = gpd.sjoin(df_gps, df_stores_geo, how="left", op='within')
    
    #drop the rows where store_is is empty
    gps_signals_w_stores= gps_signals_w_stores[~gps_signals_w_stores['store_id'].isna()]
    gps_signals_w_stores.drop('index_right', axis=1, inplace=True)
    
    #append filtered gps signals to df_gps_signals_w_stores
    df_gps_signals_w_stores = df_gps_signals_w_stores.append(gps_signals_w_stores)




100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [03:59<00:00,  4.20s/it]


In [6]:
df_gps_signals_w_stores.head()

,device_id,date,geometry,store_id,store_name
1709,6790,2021-01-01,POINT (13.54715 52.43185),place_25,McDonald's
3199,13577,2021-01-01,POINT (13.38807 52.51999),place_29,McDonald's
7649,8549,2021-01-01,POINT (13.24398 52.61495),place_141,Rewe
9764,9288,2021-01-01,POINT (13.54691 52.43192),place_25,McDonald's
13247,7144,2021-01-01,POINT (13.59309 52.44798),place_18,McDonald's


### 2.3. Affinitiy Data

In [7]:

path = r'C:\Users\senol_c1\challenge\assignment_data\affinities'            
all_files = glob.glob(path + "/*")

#read affinity files into dataframe
df_affinities = pd.DataFrame()
for filename in all_files:
    #read affinity files into dataframe and set device_id as header 
    df = pd.read_csv(filename, index_col=None, names=['device_id'])
    df['device_id'] = df['device_id'].astype(int)
    
    #split file name to get affinity name
    affinity_name =  filename.split('\\')[-1]
    
    #assign 1 to device if it belongs to an affinity 
    df[affinity_name] = 1
    
    #if dataframe is empty assign dataframe to itself
    if df_affinities.shape[0] == 0:
        df_affinities = df
        
    #if dataframe is not empty merge dataframess to create final affinity data
    else:
        df_affinities = df_affinities.merge(df, on='device_id', how='outer')
        
#fill empty values with 0        
df_affinities = df_affinities.fillna(0)

In [8]:
df_affinities.head()

,device_id,addidas,apple,bmw,employed,female,high_income,honda,h_&_m,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,50844,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,90618,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,97745,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,45843,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,164807,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## 3. Data Processing

### 3.1. Total number of GPS signals per place_id/date

In [9]:
#create device_id_count 
df_gps_signals_w_stores['device_id_count'] = df_gps['device_id'].copy()
df_gps_signals_w_stores.head()

,device_id,date,geometry,store_id,store_name,device_id_count
1709,6790,2021-01-01,POINT (13.54715 52.43185),place_25,McDonald's,36226
3199,13577,2021-01-01,POINT (13.38807 52.51999),place_29,McDonald's,42672
7649,8549,2021-01-01,POINT (13.24398 52.61495),place_141,Rewe,24391
9764,9288,2021-01-01,POINT (13.54691 52.43192),place_25,McDonald's,33170
13247,7144,2021-01-01,POINT (13.59309 52.44798),place_18,McDonald's,57916


In [10]:
#calculate each users visits per 'store_id','store_name','date'
df_gps_visits = df_gps_signals_w_stores.groupby(by= ['device_id','store_id','store_name','date'])[['device_id_count']].count().reset_index()
df_gps_visits.head()

,device_id,store_id,store_name,date,device_id_count
0,235,place_193,Rewe,2021-01-04,1
1,268,place_141,Rewe,2021-01-09,1
2,288,place_213,Kaufland,2021-01-09,4
3,343,place_156,Rewe,2021-01-09,3
4,370,place_119,Aldi,2021-01-09,2


In [11]:
#sum device_id_counts of users per store_id, store_name and date to get total_signals
df_total_signals = df_gps_visits.groupby(by= ['store_name','store_id','date',])[['device_id_count']].sum().reset_index()
df_total_signals.head()

,store_name,store_id,date,device_id_count
0,Aldi,place_101,2021-01-04,1
1,Aldi,place_101,2021-01-05,1
2,Aldi,place_102,2021-01-02,3
3,Aldi,place_102,2021-01-04,1
4,Aldi,place_102,2021-01-06,2


### 3.2. Total number of unique visitors (i.e. device ids)

In [15]:
#count number of uniqu device_idsper store_id, storename and date
df_unique_visits = df_gps_visits.groupby(by= ['store_id','store_name','date'])[['device_id']].nunique().reset_index()
df_unique_visits.columns = ['store_id','store_name','date', 'unique_visits']
df_unique_visits.head()

,store_id,store_name,date,unique_visits
0,place_1,McDonald's,2021-01-04,1
1,place_1,McDonald's,2021-01-06,1
2,place_1,McDonald's,2021-01-08,1
3,place_1,McDonald's,2021-01-12,1
4,place_1,McDonald's,2021-01-16,1


### 3.3. Total number of unique visitors belonging to each affinity group

In [16]:
#merge affinity data with unique visitors data to get find affinitiy of unique visitors
df_user = df_gps_visits.merge(df_affinities, on='device_id', how='left')
df_user.head()

,device_id,store_id,store_name,date,device_id_count,addidas,apple,bmw,employed,female,high_income,honda,h_&_m,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,235,place_193,Rewe,2021-01-04,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,268,place_141,Rewe,2021-01-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,288,place_213,Kaufland,2021-01-09,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,343,place_156,Rewe,2021-01-09,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,370,place_119,Aldi,2021-01-09,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#sum affinitiy values in each affinitiy columns per 'store_name','store_id','date'
    #to get number of unique visitors belonging to each afiinitiy group
df_affinity_sum = df_user.groupby(by= ['store_name','store_id','date'])[[ 'apple', 'tommy_helfinger',
       'middle_income', 'student', 'addidas', 'retired', 'high_income', 'male',
       'employed', 'low_income', 'job_seeking', 'bmw', 'mercedes-benz',
       'female', 'h_&_m', 'honda']].sum().reset_index()
df_affinity_sum.head()

,store_name,store_id,date,apple,tommy_helfinger,middle_income,student,addidas,retired,high_income,male,employed,low_income,job_seeking,bmw,mercedes-benz,female,h_&_m,honda
0,Aldi,place_101,2021-01-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Aldi,place_101,2021-01-05,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aldi,place_102,2021-01-02,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aldi,place_102,2021-01-04,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aldi,place_102,2021-01-06,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Final dataset as in the examaple.csv

In [19]:
#merge dataframes to get the final data as in the exmaple.csv
# merge total_signals dataframe with unique_visits dataframe
df_final_temp = df_total_signals.merge(df_unique_visits, on= ['store_id','date', 'store_name'], how='left')
df_final_temp.head()

,store_name,store_id,date,device_id_count,unique_visits
0,Aldi,place_101,2021-01-04,1,1
1,Aldi,place_101,2021-01-05,1,1
2,Aldi,place_102,2021-01-02,3,1
3,Aldi,place_102,2021-01-04,1,1
4,Aldi,place_102,2021-01-06,2,2


In [20]:
# merge resulting dataframe with affinitiy_sum dataframe
df_final = df_final_temp.merge(df_affinity_sum, on= ['store_id','date', 'store_name'], how='left')
df_final = df_final.rename(columns= {'device_id_count':'total_signals'})
df_final.head()

,store_name,store_id,date,total_signals,unique_visits,apple,tommy_helfinger,middle_income,student,addidas,retired,high_income,male,employed,low_income,job_seeking,bmw,mercedes-benz,female,h_&_m,honda
0,Aldi,place_101,2021-01-04,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Aldi,place_101,2021-01-05,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aldi,place_102,2021-01-02,3,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aldi,place_102,2021-01-04,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aldi,place_102,2021-01-06,2,2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 5.Output csv file

In [21]:
# Write final dataframe to csv
df_final.to_csv(r'sample_data_analysis_result.csv',index=False)
